## Download Yelp business POI using the API

In [1]:
"""
Yelp Fusion API code sample.

This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.

Please refer to https://docs.developer.yelp.com/docs/get-started for the API
documentation.
"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
        #'Content - Language': 'en - US',
    }

    #print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):  # 
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        business_id (str): The search business_id passed to the API.

    Returns:
        dict: The JSON response from the request.
    """
    url_params = {
        'location': location.replace(' ', '+'),
        'limit': search_limit  # ,
        #'offset': 1000
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(api_key, term, location, output_file):  # 
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(api_key, term, location)  #

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses in {0} found.'.format(location))
        return
    
    print(u'Searching for {0} in {1}; {2} businesses found; Output json file will be saved to {3}'.format(search_term,location, len(businesses), output_file))
    
    for business in businesses:
        business_id = business['id']
        print(u'\t querying business info for the result of business ID: "{0}" ...'.format(business_id))
        '''
        business_id = businesses[0]['id']

        print(u'{0} businesses found, querying business info '
              'for the top result "{1}" ...'.format(
                  len(businesses), business_id))
        '''
        response = get_business(api_key, business_id)
        #print(u'Result for business "{0}" found:'.format(business_id))
        #pprint.pprint(response, indent=2)

        # Save the response to a JSON file
        with open(output_file, 'a') as json_file:
            json.dump(response, json_file)
            json_file.write("\n")


In [2]:
# Set your API key
api_key = "paste_your_yelp_api_key_here"
search_term = 'restaurant' 
location = 'Athens, GA'
json_path = 'business.json'
search_limit = 50

In [3]:
try:
    query_api(api_key,search_term, location, json_path) 
except HTTPError as error:
    sys.exit('Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
            error.code,
            error.url,
            error.read(),))

Searching for restaurant in Athens, GA; 50 businesses found; Output json file will be saved to business.json
	 querying business info for the result of business ID: "PTkAWaOk61zh1DRjUGUfcQ" ...
	 querying business info for the result of business ID: "PAfHPF4ea4fprm_Ji_ILxA" ...
	 querying business info for the result of business ID: "Upa0qxjMbVEaNTmpvTVANw" ...
	 querying business info for the result of business ID: "sEMCF4SI2RCzG7pBYr3Nsg" ...
	 querying business info for the result of business ID: "lMgGXIEU_4pkFpqdAwHGZw" ...
	 querying business info for the result of business ID: "10UVxiLX4eZZeypz8ffCwQ" ...
	 querying business info for the result of business ID: "XqUShUXJUSDPNhwXzqbqMg" ...
	 querying business info for the result of business ID: "db0uHZi_oRcevHqBD9pWKw" ...
	 querying business info for the result of business ID: "kif84V4moSrLUXNfZIJ03Q" ...
	 querying business info for the result of business ID: "A4gYyLPDHJxdU2Q4aZ2qIQ" ...
	 querying business info for the result o

# Read the output json file

## Preparing Data
In this section, we define the input parameters that our Moran's I tool will need. This includes:
- The path to the feature class which contains our geographic data.
- The name of the field that holds the attribute data we wish to analyze (in this case, 'rating').
- The path where the output report will be saved.

## Importing Modules
In this cell, we are importing necessary Python modules that are required for our spatial analysis.
- `arcpy`: The primary module for ArcGIS automation, which allows us to perform GIS tasks.
- `arcpy.sa`: A submodule of `arcpy` for spatial analysis tasks.
- `os`: A module that provides functions to interact with the operating system.

## Setting the Workspace
Here we set the workspace environment for ArcPy. The workspace is the foundational setting that establishes the working directory for any files that our script will use or produce. By setting it once at the beginning, we ensure that all subsequent operations look for files in the right place without needing to specify full paths.

In [35]:
import pandas as pd
json_path= json_path
csv_path = 'business.csv'
df = pd.read_json(json_path, lines=True)
df = df.drop_duplicates(subset='id', keep='first')

# Read the lon, lat columns
df['latitude'] = df['coordinates'].apply(lambda x: x['latitude'])
df['longitude'] = df['coordinates'].apply(lambda x: x['longitude'])

df

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,rating,location,coordinates,photos,price,hours,transactions,messaging,latitude,longitude
0,Jcds-gbFMQJ36wPHFeHzIw,mamas-boy-at-the-falls-athens,Mama's Boy at the Falls,https://s3-media3.fl.yelpcdn.com/bphoto/75OoM8...,True,False,https://www.yelp.com/biz/mamas-boy-at-the-fall...,17068508550,(706) 850-8550,165,...,4.0,"{'address1': '8851 Macon Hwy', 'address2': 'St...","{'latitude': 33.9095495614517, 'longitude': -8...",[https://s3-media3.fl.yelpcdn.com/bphoto/75OoM...,$$,"[{'open': [{'is_overnight': False, 'start': '0...",[],NaN,33.909550,-83.402044
1,PTkAWaOk61zh1DRjUGUfcQ,last-resort-grill-athens,Last Resort Grill,https://s3-media4.fl.yelpcdn.com/bphoto/rOel6i...,True,False,https://www.yelp.com/biz/last-resort-grill-ath...,17065490810,(706) 549-0810,814,...,4.0,"{'address1': '174-184 W Clayton St', 'address2...","{'latitude': 33.958168, 'longitude': -83.378501}",[https://s3-media4.fl.yelpcdn.com/bphoto/rOel6...,$$,"[{'open': [{'is_overnight': False, 'start': '1...",[delivery],NaN,33.958168,-83.378501
2,PAfHPF4ea4fprm_Ji_ILxA,trappeze-pub-athens,Trappeze Pub,https://s3-media1.fl.yelpcdn.com/bphoto/eIcsIT...,True,False,https://www.yelp.com/biz/trappeze-pub-athens?a...,17065438997,(706) 543-8997,501,...,4.0,"{'address1': '269 N Hull St', 'address2': 'Ste...","{'latitude': 33.9586192714609, 'longitude': -8...",[https://s3-media1.fl.yelpcdn.com/bphoto/eIcsI...,$$,"[{'open': [{'is_overnight': False, 'start': '1...",[delivery],NaN,33.958619,-83.378935
3,Upa0qxjMbVEaNTmpvTVANw,the-place-athens,The Place,https://s3-media2.fl.yelpcdn.com/bphoto/0Gex4A...,True,False,https://www.yelp.com/biz/the-place-athens?adju...,17068502988,(706) 850-2988,454,...,4.5,"{'address1': '229 E Broad St', 'address2': '',...","{'latitude': 33.95806332512856, 'longitude': -...",[https://s3-media2.fl.yelpcdn.com/bphoto/0Gex4...,$$,"[{'open': [{'is_overnight': False, 'start': '1...","[delivery, pickup]",NaN,33.958063,-83.375037
4,sEMCF4SI2RCzG7pBYr3Nsg,south-kitchen-bar-athens-2,South Kitchen + Bar,https://s3-media3.fl.yelpcdn.com/bphoto/l59XRF...,True,False,https://www.yelp.com/biz/south-kitchen-bar-ath...,17063956125,(706) 395-6125,405,...,4.0,"{'address1': '247 E Washington St', 'address2'...","{'latitude': 33.9596729824125, 'longitude': -8...",[https://s3-media3.fl.yelpcdn.com/bphoto/l59XR...,$$,"[{'open': [{'is_overnight': False, 'start': '1...",[delivery],NaN,33.959673,-83.375258
5,10UVxiLX4eZZeypz8ffCwQ,seabear-oyster-bar-athens,Seabear Oyster Bar,https://s3-media3.fl.yelpcdn.com/bphoto/lOK48G...,True,False,https://www.yelp.com/biz/seabear-oyster-bar-at...,17068504367,(706) 850-4367,250,...,4.5,"{'address1': '297 Prince Ave', 'address2': 'St...","{'latitude': 33.960074117071194, 'longitude': ...",[https://s3-media3.fl.yelpcdn.com/bphoto/lOK48...,$$,"[{'open': [{'is_overnight': False, 'start': '1...",[delivery],NaN,33.960074,-83.383061
6,XqUShUXJUSDPNhwXzqbqMg,the-table-bistro-athens,The Table Bistro,https://s3-media2.fl.yelpcdn.com/bphoto/LPXvYE...,True,False,https://www.yelp.com/biz/the-table-bistro-athe...,17063956599,(706) 395-6599,186,...,4.5,"{'address1': '1075 Baxter St', 'address2': 'St...","{'latitude': 33.9464243857486, 'longitude': -8...",[https://s3-media2.fl.yelpcdn.com/bphoto/LPXvY...,$$,"[{'open': [{'is_overnight': False, 'start': '1...",[delivery],NaN,33.946424,-83.400187
7,db0uHZi_oRcevHqBD9pWKw,the-royal-peasant-athens,The Royal Peasant,https://s3-media2.fl.yelpcdn.com/bphoto/BtLvSX...,True,False,https://www.yelp.com/biz/the-royal-peasant-ath...,17065497920,(706) 549-7920,259,...,4.5,"{'address1': '1675 S Lumpkin St', 'address2': ...","{'latitude': 33.93799428735798, 'longitude': -...",[https://s3-media2.fl.yelpcdn.com/bphoto/BtLvS...,$$,"[{'open': [{'is_overnight': False, 'start': '1...",[delivery],NaN,33.937994,-83.387036
8,lMgGXIEU_4pkFpqdAwHGZw,mamas-boy-athens,Mama's Boy,https://s3-media4.fl.yelpcdn.com/bphoto/we5zTZ...,True,False,https://www.yelp

# Use arcpy for spatial analysis

### Create business POI feature class from the Yelp data

In [9]:
import pandas as pd
import arcpy

# Assuming df is your pandas DataFrame with 'latitude' and 'longitude' columns

# Set the environment settings
arcpy.env.workspace = 'path_to_your_workspace'  # Replace with your workspace path
arcpy.env.overwriteOutput = 1

# Define a path for the new feature class
out_fc = 'business' # Replace with your file name

# Create an empty Point feature class
spatial_reference = arcpy.SpatialReference(4326)  # WGS 84
arcpy.management.CreateFeatureclass(arcpy.env.workspace, out_fc, 'POINT', '', '', '', spatial_reference)

# Use an insert cursor to add points
fields = ['SHAPE@XY', 'rating']

# Add the fields to the feature class
for field_name in fields[1:]:
    arcpy.management.AddField(out_fc, field_name, 'DOUBLE')  # You can change the field type as needed

# Open an InsertCursor to add points and attributes
with arcpy.da.InsertCursor(out_fc, fields) as cursor:
    for index, row in df.iterrows():
        # Extract values from the DataFrame row
        longitude = row['longitude']
        latitude = row['latitude']
        rating = row['rating']

        # Create a point geometry
        point = arcpy.Point(longitude, latitude)

        # Create a feature with the point geometry and attribute values
        feature = (point, rating)
        # Insert the feature into the feature class
        cursor.insertRow(feature)


### Moran's I on the business data
The Moran's I tool measures spatial autocorrelation based on feature locations and attribute values. We are running the tool with several parameters that specify how the spatial relationships are conceptualized (in this case, using an inverse distance method), the method for measuring distances (Euclidean), and how the data should be standardized.

In [34]:
# Check out any necessary licenses
arcpy.CheckOutExtension("spatial")

# Local variables:
input_feature_class = 'business.shp'  # Replace with your shapefile
input_field = 'rating'  # Replace with your field of interest

# Execute Spatial Autocorrelation (Moran's I)
Index, ZScore, PValue, Report_File = arcpy.stats.SpatialAutocorrelation(
    Input_Feature_Class=input_feature_class,
    Input_Field=input_field,
    Generate_Report="GENERATE_REPORT", 
    Conceptualization_of_Spatial_Relationships="INVERSE_DISTANCE", 
    Distance_Method="EUCLIDEAN_DISTANCE", 
    Standardization="ROW"
)

# Check in any necessary licenses
arcpy.CheckInExtension("spatial")

print(f"Moran's I Analysis Finished!\nIndex: {Index}\nZScore: {ZScore}\nPValue: {PValue}\nReport File: {Report_File}")

Moran's I Analysis Finished!
Index: 0.107773
ZScore: 2.098428
PValue: 0.035867
Report File: C:\Users\54475\AppData\Local\Temp\MoransI_Result_20152_18740_.html
